In [1]:
from Dynamics import Dynamics, np, plt
import pandas as pd

### Modeling the US Labor Market
###### Updated August 2020

In [2]:
df = pd.read_csv('/Users/grant/Jupyter/Dynamics/Labor Market Data.csv')
df.columns = ['date', 'pop', 'employed', 'unemployed', 'eu','ue','ne','un', 'en', 'nu']

In [3]:
df

,date,pop,employed,unemployed,eu,ue,ne,un,en,nu
0,1990-01-01,248743.0,119081.0,7413.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-02-01,248920.0,119059.0,7296.0,1965.0,2377.0,3006.0,1410.0,3319.0,1642.0
2,1990-03-01,249146.0,119203.0,6852.0,1578.0,1999.0,3152.0,1472.0,2777.0,1448.0
3,1990-04-01,249436.0,118852.0,6620.0,1602.0,1884.0,2975.0,1353.0,2945.0,1423.0
4,1990-05-01,249707.0,119151.0,6533.0,1478.0,1906.0,3162.0,1257.0,2735.0,1578.0
...,...,...,...,...,...,...,...,...,...,...
387,2022-04-01,332863.0,158105.0,5458.0,1490.0,1932.0,4323.0,1675.0,4880.0,1400.0
388,2022-05-01,332928.0,158426.0,5548.0,1312.0,1600.0,4593.0,1328.0,4309.0,1699.0
389,2022-06-01,333028.0,158111.0,6334.0,1748.0,1724.0,5516.0,1377.0,5454.0,2128.0
390,2022-07-01,333137.0,158290.0,6255.0,1655.0,1838.0,5077.0,1693.0,4792.0,1789.0


In [4]:
# creating coefficients for movement as 
df['EU'] = df['eu']/df['employed']
df['UE'] = df['ue'] / df['unemployed']
df['NE'] = df['ne']/(df['employed']+df['unemployed'])
df['UN'] = df['un'] / df['unemployed']
df['EN'] = df['en'] / df['employed']
df['NU'] = df['nu']/ (df['employed']+df['unemployed'])
df['OOLF'] = df['pop'] - df['employed'] - df['unemployed']
df = df.iloc[:-1]

_____________________________________

#### Predicting August 2022 Results
##### 24 month moving average

In [5]:
# setting our backwards average length in months, and forward predict length in months

forward_pred = 1

back_avg = 24

# defining our coefficients as historical moving averages

UE = df['UE'].iloc[-back_avg:].mean()
NE = df['NE'].iloc[-back_avg:].mean()
EU = df['EU'].iloc[-back_avg:].mean()
EN = df['EN'].iloc[-back_avg:].mean()
NU = df['NU'].iloc[-back_avg:].mean()
UN = df['UN'].iloc[-back_avg:].mean()

# defining functions based

def e(e, u, n):
    return UE*u + NE*n + (1 - EU + EN)*e

def u(e, u, n):
    return EU*e + NU*n + (1 - UE + UN)*u

def n(e, u, n):
    return EN*e + UN*u + (1 - NE + NU)*n

# Instantiating Dynamics object with a list of functions and initial_conditions

initial_conditions = [df['employed'].iloc[-1], 
                      df['unemployed'].iloc[-1], 
                      df['OOLF'].iloc[-1]
                     ]

functions = [e, u, n]

dynamics = Dynamics(functions, 
                    initial_conditions
                    )


# Calling the recurse() function to model the dynamics

dynamics.recurse(divergent_length=forward_pred,
                 accuracy = 0.00001 # for convergence
                )

print('Employed:', round(1000*dynamics.X[0][forward_pred], 2), 
      ' - Unemployed:', round(1000*dynamics.X[1][forward_pred], 2),
      ' - Not in Labor Force:', round(1000*dynamics.X[2][forward_pred],2))

#dynamics.trivariate_plot(figsize = (15,15))

Employed: 167693107.3  - Unemployed: 10116777.84  - Not in Labor Force: 173400666.45


_____________________________________

#### Predicting September 2022 Results
##### 24 month moving average

In [6]:
# setting our backwards average length in months, and forward predict length in months

forward_pred = 2

back_avg = 24

UE = df['UE'].iloc[-back_avg:].mean()
NE = df['NE'].iloc[-back_avg:].mean()
EU = df['EU'].iloc[-back_avg:].mean()
EN = df['EN'].iloc[-back_avg:].mean()
NU = df['NU'].iloc[-back_avg:].mean()
UN = df['UN'].iloc[-back_avg:].mean()

# defining functions based on historical coefficients
def e(e, u, n):
    return UE*u + NE*n + (1 - EU + EN)*e

def u(e, u, n):
    return EU*e + NU*n + (1 - UE + UN)*u

def n(e, u, n):
    return EN*e + UN*u + (1 - NE + NU)*n

# Instantiating Dynamics object with a list of functions and initial_conditions

initial_conditions = [df['employed'].iloc[-1], df['unemployed'].iloc[-1], df['OOLF'].iloc[-1]]
functions = [e, u, n]

dynamics = Dynamics(functions, 
                    initial_conditions
                    )


# Calling the model() function to model the dynamics until we've converged our diverged.


dynamics.recurse(divergent_length=forward_pred,
                 accuracy = 0.00001 # for convergence
                )

print('Employed:', round(1000*dynamics.X[0][forward_pred], 2), 
      ' - Unemployed:', round(1000*dynamics.X[1][forward_pred], 2),
      ' - Not in Labor Force:', round(1000*dynamics.X[2][forward_pred],2))

#dynamics.trivariate_plot(figsize=(15,15))

Employed: 178467698.33  - Unemployed: 14015077.7  - Not in Labor Force: 179379024.08


_____________________________________

#### Predicting August 2022 Results
##### 120 month moving average

In [7]:
# setting our backwards average length in months, and forward predict length in months

forward_pred = 2

back_avg = 120

UE = df['UE'].iloc[-back_avg:].mean()
NE = df['NE'].iloc[-back_avg:].mean()
EU = df['EU'].iloc[-back_avg:].mean()
EN = df['EN'].iloc[-back_avg:].mean()
NU = df['NU'].iloc[-back_avg:].mean()
UN = df['UN'].iloc[-back_avg:].mean()

# defining functions based on historical coefficients
def e(e, u, n):
    return UE*u + NE*n + (1 - EU + EN)*e

def u(e, u, n):
    return EU*e + NU*n + (1 - UE + UN)*u

def n(e, u, n):
    return EN*e + UN*u + (1 - NE + NU)*n

# Instantiating Dynamics object with a list of functions and initial_conditions

initial_conditions = [df['employed'].iloc[-1], df['unemployed'].iloc[-1], df['OOLF'].iloc[-1]]
functions = [e, u, n]

dynamics = Dynamics(functions, 
                    initial_conditions
                    )


# Calling the model() function to model the dynamics until we've converged our diverged.


dynamics.recurse(divergent_length=forward_pred,
                 accuracy = 0.00001 # for convergence
                )

print('Employed:', round(1000*dynamics.X[0][forward_pred], 2), 
      ' - Unemployed:', round(1000*dynamics.X[1][forward_pred], 2),
      ' - Not in Labor Force:', round(1000*dynamics.X[2][forward_pred],2))

#dynamics.trivariate_plot(figsize=(15,15))

Employed: 177182363.96  - Unemployed: 15011414.67  - Not in Labor Force: 180091121.74
